<center><h2>Лабораторна робота 2<h2></center>
Виконав: ФБ-21 Новіцький Олександр

Створити env в якому будуть встановлені всі необхідні бібліотеки та налаштування для данної лабораторної роботи.

In [1]:
import datetime
import os
import pandas as pd
import urllib.request

print("Success")

Success


Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу.

Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області.

При зберіганні файлу до його імені потрібно додати дату та час завантаження.

In [4]:

CURRENT_DATE = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

#DIR_NAME = f"vhi_data_{CURRENT_DATE}"
DIR_NAME = "vhi_data_15-03-2024_23-19-19"

#os.makedirs(DIR_NAME, exist_ok=True)

def getFile(pid, start=1981, stop=2020):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={pid}&year1={start}&year2={stop}&type=Mean"
    response = urllib.request.urlopen(url)

    path = os.path.join(DIR_NAME, f"vhi_{pid}_{CURRENT_DATE}.csv")
    
    with open(path, 'wb') as output:
        output.write(response.read())

for i in range(1, 28):
    #getFile(i)
    pass

Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).

Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо).

Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [5]:
files = os.listdir(DIR_NAME)


headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']

res = []


def loadFile(file):
    pid = int(file.split('_')[1])

    df = pd.read_csv(os.path.join(DIR_NAME, file), header=1, names=headers)

    # Відкинути всі nan. При цьому враховуйте, що nan у файлі позначаються як -1
    df = df.drop(df.loc[df['VHI'] == -1].index)
    # Відкинути пустий стовбчик
    df = df.drop("empty", axis=1)
    # Відкинути останній рядок
    df = df.drop(df.index[-1])
    # В першому рядку неправильно задано рік
    df.at[0, "Year"] = df.at[0, "Year"][9:]
    # Дропнути Na
    df = df.dropna()

    df.insert(0, "pID", pid, True)
    

    res.append(df)



for file in files:
    loadFile(file)

finalDf = pd.concat(res).drop_duplicates().reset_index(drop=True)
finalDf = finalDf.loc[(finalDf.pID != 12) & (finalDf.pID != 20)]

print(finalDf)

       pID  Year  Week    SMN     SMT    VCI    TCI    VHI
0       10  1982   1.0  0.059  258.24  51.11  48.78  49.95
1       10  1982   2.0  0.063  261.53  55.89  38.20  47.04
2       10  1982   3.0  0.063  263.45  57.30  32.69  44.99
3       10  1982   4.0  0.061  265.10  53.96  28.62  41.29
4       10  1982   5.0  0.058  266.42  46.87  28.57  37.72
...    ...   ...   ...    ...     ...    ...    ...    ...
53401    9  2020  48.0  0.159  277.21  66.52  16.25  41.37
53402    9  2020  49.0  0.150  275.06  65.68  19.44  42.55
53403    9  2020  50.0  0.144  274.00  66.17  18.69  42.43
53404    9  2020  51.0  0.138  272.73  66.79  19.59  43.19
53405    9  2020  52.0  0.132  271.44  67.24  21.90  44.57

[49450 rows x 8 columns]


In [8]:
print(finalDf.isna().sum())

pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64


Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні.

In [10]:
def changePID(df):
    provinces = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }

    df['pID'] = df['pID'].replace(provinces)
    return df

finalDf = changePID(finalDf)



print(finalDf)


pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
       pID  Year  Week    SMN     SMT    VCI    TCI    VHI
0       17  1982   1.0  0.059  258.24  51.11  48.78  49.95
1       17  1982   2.0  0.063  261.53  55.89  38.20  47.04
2       17  1982   3.0  0.063  263.45  57.30  32.69  44.99
3       17  1982   4.0  0.061  265.10  53.96  28.62  41.29
4       17  1982   5.0  0.058  266.42  46.87  28.57  37.72
...    ...   ...   ...    ...     ...    ...    ...    ...
53401   20  2020  48.0  0.159  277.21  66.52  16.25  41.37
53402   20  2020  49.0  0.150  275.06  65.68  19.44  42.55
53403   20  2020  50.0  0.144  274.00  66.17  18.69  42.43
53404   20  2020  51.0  0.138  272.73  66.79  19.59  43.19
53405   20  2020  52.0  0.132  271.44  67.24  21.90  44.57

[49450 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
- Ряд VHI для області за рік, пошук екстремумів (min та max);

In [11]:
def extremes(df, start=1981, stop=2020):
    result = []
    df = df[df['Year'].astype(int) >= start]
    df = df[df['Year'].astype(int) <= stop]

    pID_values = df['pID']

    pid_values = pID_values.unique()

    year_values = df['Year']

    year_values = year_values.unique()

    for year in year_values:
        year = int(year)

        for pid in pid_values:
            new_df = df[(df['Year'].astype(int) == year) & (df['pID'] == pid)].copy()
            new_df = new_df.dropna()

            new_df.reset_index(drop=True, inplace=True)
            min_vhi = new_df['VHI'].min()
            max_vhi = new_df['VHI'].max()

            result.append({'Year': year, 'pID': pid, 'Max_VHI': max_vhi, 'Min_VHI': min_vhi})

    return result


extremes_df = pd.DataFrame(extremes(finalDf, start=2000, stop=2024))

print(extremes_df)


pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0
VHI     0
dtype: int64
pID     0
Year    0
Week    0
SMN     0
SMT     0
VCI     0
TCI     0


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

- Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка областей;

- Аналогічно для помірних посух

In [46]:
def drought(df, percentage, vhi_start=0, vhi_end=15, start=1981, stop=2020):
    year_values = df['Year']

    year_values = year_values.unique()

    pID_values = df['pID']

    pid_values = pID_values.unique()

    total_oblasts = len(pid_values)

    for year in year_values:
        year = int(year)
        new_df = df[(df['Year'].astype(int) == year)].copy()
        target_oblasts = new_df[(new_df['VHI'] >= vhi_start) & ( new_df['VHI'] < vhi_end)]['pID'].nunique()
        
        res_percentage = target_oblasts / total_oblasts * 100

        if res_percentage >= percentage:
            print(
                f"In year {year} there were {target_oblasts} oblasts with vhi\
                      from {vhi_start} to {vhi_end}, which is {res_percentage}% of all"
            )
        

        #target_oblasts = target_oblasts.uniq

drought(finalDf, 10, vhi_start=0, vhi_end=15)
print('-'*100)
drought(finalDf, 10, vhi_start=15, vhi_end=35)

In year 2000 there were 4 oblasts with vhi from 0 to 15, which is 16.0% of all
In year 2007 there were 5 oblasts with vhi from 0 to 15, which is 20.0% of all
----------------------------------------------------------------------------------------------------
In year 1982 there were 25 oblasts with vhi from 15 to 35, which is 100.0% of all
In year 1983 there were 25 oblasts with vhi from 15 to 35, which is 100.0% of all
In year 1984 there were 24 oblasts with vhi from 15 to 35, which is 96.0% of all
In year 1985 there were 24 oblasts with vhi from 15 to 35, which is 96.0% of all
In year 1986 there were 25 oblasts with vhi from 15 to 35, which is 100.0% of all
In year 1987 there were 18 oblasts with vhi from 15 to 35, which is 72.0% of all
In year 1988 there were 22 oblasts with vhi from 15 to 35, which is 88.0% of all
In year 1989 there were 24 oblasts with vhi from 15 to 35, which is 96.0% of all
In year 1990 there were 23 oblasts with vhi from 15 to 35, which is 92.0% of all
In year 1